In [ ]:
# -*- coding: utf-8 -*- """ Generate predictions (XGB/MF/GGM + blend) and export metrics (MAE, RMSE, R2) to Excel. """ from __future__ import annotations from pathlib import Path import json, joblib, warnings import numpy as np import pandas as pd from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score # ========================= # 0) PARAMETERS — EDIT HERE # ========================= # Artifacts MODELS_DIR = Path("models_streamlit_xgb") # chứa rf_model_{safe_name}.joblib INDEX_CSV = Path("index.csv") # sổ tra cứu target,K -> model_path SCALER_P = Path("2/scaler.joblib") # scaler chuẩn hoá SUBJECTS_P = Path("3/subjects.json") # danh sách/trật tự môn MF_PATH = Path("models_streamlit_mf/find-subject-score.joblib") # MF artifacts GGM_PATH = Path("models_streamlit_ggm/ggm.joblib") # GGM artifacts # Data DATA_XLSX = Path("Data_clean/Data_subject_complete.xlsx") # Excel gốc EVAL_SPLIT = "test" # 'train' | 'val' | 'test' | hoặc list/tuple như ('val','test') # Output PREDS_CSV = Path("preds_generated.csv") METRICS_XLSX = Path("metrics.xlsx") # Control ONLY_SUBJECTS: list[str] | None = None # ví dụ ["Giải tích I", "Đại số"]; None = dùng toàn bộ SUBJECTS_P MAX_ROWS: int | None = None # giới hạn hàng để chạy nhanh; None = tất cả MIN_K = 3 # bỏ qua case có < MIN_K môn đã biết (trừ target) EPS = 1e-6 # epsilon số học warnings.filterwarnings("ignore", category=RuntimeWarning) # ========================= # 1) LOAD ARTIFACTS & DATA # ========================= def load_subjects(path: Path) -> list[str]: with open(path, "r", encoding="utf-8") as f: obj = json.load(f) if isinstance(obj, dict) and "subjects" in obj: return list(obj["subjects"]) if isinstance(obj, list): return list(obj) raise ValueError("SUBJECTS_P must be a JSON list or a dict with key 'subjects'.") def load_excel(path: Path) -> pd.DataFrame: df = pd.read_excel(path) need = {"EncryptedID", "split"} if not need.issubset(df.columns): raise ValueError(f"Excel must include {need}. Found: {list(df.columns)}") return df def load_scaler(path: Path, train_df: pd.DataFrame, subjects: list[str]) -> tuple[dict, dict]: """Load means/stds (từ joblib dạng dict hoặc StandardScaler). Fallback: tính từ train_df.""" means, stds, ok = {}, {}, False if path.exists(): try: scl = joblib.load(path) if isinstance(scl, dict): if "means" in scl and "stds" in scl: means = {k: float(v) for k, v in scl["means"].items()} stds = {k: float(v) for k, v in scl["stds"].items()} ok = True elif "mean" in scl and "scale" in scl: means = {s: float(m) for s, m in zip(subjects, scl["mean"])} stds = {s: float(v if v != 0 else 1.0) for s, v in zip(subjects, scl["scale"])} ok = True else: if hasattr(scl, "mean_") and hasattr(scl, "scale_"): means = {s: float(m) for s, m in zip(subjects, scl.mean_)} stds = {s: float(v if v != 0 else 1.0) for s, v in zip(subjects, scl.scale_)} ok = True except Exception as e: print("[Scaler] load failed:", e) if not ok: sub_train = train_df[subjects] means = sub_train.mean(axis=0, skipna=True).to_dict() stds = sub_train.std(axis=0, ddof=0, skipna=True).replace(0, 1.0).to_dict() means = {k: float(v) for k, v in means.items()} stds = {k: float(v) for k, v in stds.items()} print("⚠️ Using fallback means/stds computed from train split.") return means, stds def load_xgb_index(path: Path) -> pd.DataFrame: idx = pd.read_csv(path) need = {"target", "K", "model_path"} if not need.issubset(idx.columns): raise ValueError(f"index.csv must include columns {need}. Found: {list(idx.columns)}") idx["model_path"] = idx["model_path"].astype(str) return idx def resolve_model_path(model_path_str: str, base_dir: Path) -> Path: p = Path(model_path_str) if p.exists(): return p if not p.is_absolute(): cand = base_dir / p if cand.exists(): return cand return cand if 'cand' in locals() else p def load_mf(path: Path): return joblib.load(path) if path.exists() else None def load_ggm(path: Path): return joblib.load(path) if path.exists() else None # ========================= # 2) FEATURE BUILDERS # ========================= def standardize(val: float, mu: float, sd: float) -> float: sd = sd if (sd and sd != 0) else 1.0 return (val - mu) / sd def standardize_user_numeric(user_numeric: dict, means: dict, stds: dict, subjects: list[str]) -> dict: """user_numeric: {subject: value or np.nan}, trong đó TARGET đã mask=NaN -> trả z-scores (NaN nếu thiếu).""" out = {} for s in subjects: v = user_numeric.get(s, np.nan) if v is None or (isinstance(v, float) and np.isnan(v)): out[s] = np.nan else: out[s] = standardize(float(v), means.get(s, 0.0), stds.get(s, 1.0)) return out def build_xgb_features(user_numeric: dict, means: dict, stds: dict, subjects: list[str]) -> np.ndarray: """ Vector đặc trưng: [z_scores..., missing_indicators...] (len = 2 * n_subjects). z_scores: NaN -> 0; missing_indicators: 1 nếu NaN, ngược lại 0. """ z = standardize_user_numeric(user_numeric, means, stds, subjects) vals = np.array([z[s] for s in subjects], dtype=float) miss = (~np.isfinite(vals)).astype(float) vals = np.nan_to_num(vals, nan=0.0) return np.concatenate([vals, miss], axis=0) # ========================= # 3) PREDICTORS (XGB / MF / GGM) # ========================= def select_xgb_model_path(target: str, K: int, xgb_index: pd.DataFrame) -> Path | None: df = xgb_index[xgb_index["target"] == target] if df.empty: return None if (df["K"] == K).any(): mp = df[df["K"] == K]["model_path"].iloc[0] return resolve_model_path(mp, MODELS_DIR) df2 = df.assign(diff=(df["K"] - K).abs()).sort_values("diff") mp = df2.iloc[0]["model_path"] return resolve_model_path(mp, MODELS_DIR) def predict_xgb_for_target(target: str, K: int, features: np.ndarray, means: dict, stds: dict) -> float: model_path = select_xgb_model_path(target, K, xgb_index) if model_path is None or not model_path.exists(): return np.nan try: model = joblib.load(model_path) y_std = float(model.predict(features.reshape(1, -1))[0]) # model học trên thang z return y_std * stds[target] + means[target] except Exception: return np.nan def predict_mf_for_target(mf_art, means, stds, user_numeric: dict, target: str, subjects: list[str]) -> float: if mf_art is None: return np.nan subj_mf = mf_art.get("subjects", subjects) # item bias if "item_bias" in mf_art: b = np.array([mf_art["item_bias"].get(s, 0.0) for s in subj_mf], dtype=float) elif "b" in mf_art: b = np.array(mf_art["b"], dtype=float) else: b = np.zeros(len(subj_mf), dtype=float) # item factors V = None for key in ("V", "item_factors", "Q"): if key in mf_art: V = np.array(mf_art[key], dtype=float) break if V is None: return np.nan lam = float(mf_art.get("lambda", mf_art.get("reg", 1.0))) idx_map = {s: i for i, s in enumerate(subj_mf)} z = standardize_user_numeric(user_numeric, means, stds, subjects) known_idx = [i for i, s in enumerate(subjects) if np.isfinite(z[s])] known_idx_mf = [idx_map[subjects[i]] for i in known_idx if subjects[i] in idx_map] if len(known_idx_mf) == 0: return np.nan r_known = np.array([z[subjects[i]] for i in known_idx], dtype=float) V_known = V[known_idx_mf, :] b_known = b[known_idx_mf] y = r_known - b_known A = V_known.T @ V_known + lam * np.eye(V_known.shape[1]) u = np.linalg.pinv(A) @ (V_known.T @ y) if target not in idx_map: return np.nan j = idx_map[target] z_hat = float(b[j] + V[j, :].T @ u) return z_hat * stds[target] + means[target] def predict_ggm_for_target(ggm_art, means, stds, user_numeric: dict, target: str, subjects: list[str]) -> tuple[float, float | None]: if ggm_art is None: return (np.nan, None) Sigma = None for k in ("cov", "sigma", "covariance"): if k in ggm_art: Sigma = np.array(ggm_art[k], dtype=float) break if Sigma is None: return (np.nan, None) subj = ggm_art.get("subjects", subjects) idx_map = {s: i for i, s in enumerate(subj)} if target not in idx_map: return (np.nan, None) z = standardize_user_numeric(user_numeric, means, stds, subjects) obs = [i for i, s in enumerate(subjects) if np.isfinite(z[s]) and s in idx_map] if len(obs) == 0: return (np.nan, None) t = idx_map[target] O = [idx_map[subjects[i]] for i in obs] try: Sigma_OO = Sigma[np.ix_(O, O)] Sigma_TO = Sigma[np.ix_([t], O)] Sigma_OT = Sigma[np.ix_(O, [t])] Sigma_TT = Sigma[t, t] z_O = np.array([z[subjects[i]] for i in obs], dtype=float).reshape(-1, 1) inv_Sigma_OO = np.linalg.pinv(Sigma_OO) mu_cond = (Sigma_TO @ inv_Sigma_OO @ z_O).ravel()[0] var_cond = float(Sigma_TT - (Sigma_TO @ inv_Sigma_OO @ Sigma_OT).ravel()[0]) y_hat = mu_cond * stds[target] + means[target] return y_hat, max(var_cond, 0.0) except Exception: return (np.nan, None) # ========================= # 4) BLEND (XGB/MF + GGM variance) # ========================= def blend(y_xgb: float, y_mf: float, y_ggm: float, var_ggm: float | None, K: int, eps: float = EPS) -> tuple[float, tuple[float, float, float]]: # Base weights theo K if K <= 10: wx, wm = 0.85, 0.15 else: wx, wm = 0.70, 0.30 # GGM theo variance w_ggm = 0.0 if var_ggm is not None and np.isfinite(var_ggm): conf = 1.0 / (var_ggm + eps) frac = conf / (conf + 1.0) w_ggm = 0.25 * float(frac) # ≤ 0.25 # Chuẩn hoá phần còn lại cho XGB/MF remain = 1.0 - w_ggm total = wx + wm if total == 0: wx_norm = wm_norm = 0.0 else: wx_norm = remain * wx / total wm_norm = remain * wm / total parts = [("xgb", y_xgb, wx_norm), ("mf", y_mf, wm_norm), ("ggm", y_ggm, w_ggm)] valid = [(n, y, w) for n, y, w in parts if np.isfinite(y)] if not valid: return np.nan, (0.0, 0.0, 0.0) total_w = sum(w for _, _, w in valid) if total_w == 0: y = float(np.mean([y for _, y, _ in valid])) return y, (wx_norm, wm_norm, w_ggm) y = sum(w * y for _, y, w in valid) / total_w return y, (wx_norm, wm_norm, w_ggm) # ========================= # 5) METRICS HELPERS # ========================= def mae(y_true, y_pred): return float(mean_absolute_error(y_true, y_pred)) def rmse(y_true, y_pred): return float(np.sqrt(mean_squared_error(y_true, y_pred))) def r2(y_true, y_pred): return float(r2_score(y_true, y_pred)) # ========================= # 6) MAIN # ========================= if __name__ == "__main__": # Load subjects = load_subjects(SUBJECTS_P) data_df = load_excel(DATA_XLSX) # fallback scaler từ train split nếu joblib không hợp lệ train_mask = data_df["split"].astype(str).str.lower().eq("train") means, stds = load_scaler(SCALER_P, data_df[train_mask], subjects) xgb_index = load_xgb_index(INDEX_CSV) mf_art = load_mf(MF_PATH) ggm_art = load_ggm(GGM_PATH) # chọn split spl = data_df["split"].astype(str).str.lower() if isinstance(EVAL_SPLIT, (list, tuple, set)): mask = spl.isin([str(s).lower() for s in EVAL_SPLIT]) else: mask = spl.eq(str(EVAL_SPLIT).lower()) use_df = data_df.loc[mask].copy() if MAX_ROWS is not None: use_df = use_df.head(int(MAX_ROWS)).copy() # subject list thực sự dùng: giao giữa SUBJECTS_P và cột Excel excel_subjects = [c for c in use_df.columns if c not in {"EncryptedID", "split"}] subj_set = set(subjects) if ONLY_SUBJECTS is None else set(ONLY_SUBJECTS) subj_list = [s for s in subjects if s in subj_set and s in excel_subjects] print(f"Loaded subjects: {len(subjects)} | Using: {len(subj_list)} (on split={EVAL_SPLIT})") print(f"MF: {'OK' if mf_art is not None else 'None'} | GGM: {'OK' if ggm_art is not None else 'None'}") # Generate predictions records = [] for _, row in use_df.iterrows(): sid = row["EncryptedID"] user_full = {s: (None if pd.isna(row.get(s)) else float(row.get(s))) for s in subj_list} for target in subj_list: y_true = user_full.get(target, None) if y_true is None or (isinstance(y_true, float) and np.isnan(y_true)): continue # K = số môn known trừ target K = int(sum(1 for s, v in user_full.items() if s != target and v is not None and np.isfinite(v))) if K < MIN_K: continue # mask target user_numeric = dict(user_full) user_numeric[target] = np.nan # build features -> XGB feats = build_xgb_features(user_numeric, means, stds, subjects) # predict từng nhánh yx = predict_xgb_for_target(target, K, feats, means, stds) ym = predict_mf_for_target(mf_art, means, stds, user_numeric, target, subjects) yg, varg = predict_ggm_for_target(ggm_art, means, stds, user_numeric, target, subjects) # blend yh, (wx_eff, wm_eff, wg_eff) = blend(yx, ym, yg, varg, K, eps=EPS) records.append({ "EncryptedID": sid, "target": target, "K": K, "y_true": y_true, "y_pred_xgb": yx, "y_pred_mf": ym, "y_pred_ggm": yg, "y_pred_hyb": yh, "w_xgb_eff": wx_eff, "w_mf_eff": wm_eff, "w_ggm_eff": wg_eff, "var_ggm": varg, }) preds_df = pd.DataFrame.from_records(records) preds_df.to_csv(PREDS_CSV, index=False) print(f"✅ Saved predictions -> {PREDS_CSV} (rows={len(preds_df)})") if preds_df.empty: raise SystemExit("No predictions generated. Try lowering MIN_K or check artifacts/subjects.") # Overall metrics y_true_arr = preds_df["y_true"].to_numpy(dtype=float) pred_cols = [c for c in preds_df.columns if c.startswith("y_pred_")] overall_rows = [] for col in sorted(pred_cols): y_pred_arr = preds_df[col].to_numpy(dtype=float) overall_rows.append({ "Model": col.replace("y_pred_", ""), "MAE": mae(y_true_arr, y_pred_arr), "RMSE": rmse(y_true_arr, y_pred_arr), "R2": r2(y_true_arr, y_pred_arr), "N": len(preds_df) }) overall_df = pd.DataFrame(overall_rows).sort_values("Model").reset_index(drop=True) print("\n=== OVERALL ===") print(overall_df.to_string(index=False)) # Metrics by K bucket buckets = [(5,7), (8,10), (11,15), (16, None)] # >15 bucket_rows = [] K_arr = preds_df["K"].to_numpy(dtype=int) for low, high in buckets: if high is None: mask = (K_arr >= low) bucket_name = f"{low}+" else: mask = (K_arr >= low) & (K_arr <= high) bucket_name = f"{low}–{high}" if not np.any(mask): continue y_true_b = preds_df.loc[mask, "y_true"].to_numpy(dtype=float) for col in sorted(pred_cols): y_pred_b = preds_df.loc[mask, col].to_numpy(dtype=float) bucket_rows.append({ "Bucket(K)": bucket_name, "Model": col.replace("y_pred_", ""), "MAE": mae(y_true_b, y_pred_b), "RMSE": rmse(y_true_b, y_pred_b), "R2": r2(y_true_b, y_pred_b), "N": int(mask.sum()) }) by_bucket_df = pd.DataFrame(bucket_rows).sort_values(["Bucket(K)", "Model"]).reset_index(drop=True) if len(by_bucket_df): print("\n=== BY BUCKET (K) ===") print(by_bucket_df.to_string(index=False)) # Export Excel with pd.ExcelWriter(METRICS_XLSX, engine="xlsxwriter") as writer: overall_df.to_excel(writer, sheet_name="overall", index=False) if len(by_bucket_df): by_bucket_df.to_excel(writer, sheet_name="by_bucket", index=False) print(f"\n✅ Saved metrics -> {METRICS_XLSX}")

Loaded subjects: 31 | Using: 31 (on split=test)
MF: OK | GGM: OK
✅ Saved predictions -> preds_generated.csv (rows=4216)

=== OVERALL ===
  Scope Model      MAE     RMSE        R2    N
overall   ggm 0.534054 0.683083  0.468879 4216
overall   hyb 0.606855 0.760570  0.341547 4216
overall    mf 1.333658 1.764250 -2.542963 4216
overall   xgb 0.635628 0.826044  0.223300 4216

=== OVERALL (K ≥ 16) ===
Scope Model      MAE     RMSE        R2    N
 K≥16   ggm 0.534054 0.683083  0.468879 4216
 K≥16   hyb 0.606855 0.760570  0.341547 4216
 K≥16    mf 1.333658 1.764250 -2.542963 4216
 K≥16   xgb 0.635628 0.826044  0.223300 4216

✅ Saved metrics -> metrics.xlsx


In [1]:
# -*- coding: utf-8 -*-
"""
Evaluate XGB/MF/GGM + Hybrid using explicit subject-combination masks from a CSV.
- For each mask row (target, scenario, K, kept_subjects): keep only the listed subjects (except target),
  predict on all students in the chosen split, and record predictions.
- Compute MAE/RMSE/R2 by formula (NaN-safe).
- Export per-mask metrics, macro-avg (by scenario and by K<16/K≥16), micro-avg, and raw predictions.

NOTE: This version limits ONLY the number of mask rows (combinations) for faster runs.
      It does NOT limit the number of students in the chosen split, so your metrics stay representative.
"""

from __future__ import annotations
from pathlib import Path
import json, joblib, warnings, ast
import numpy as np
import pandas as pd

# =========================
# 0) CONFIG
# =========================
# Artifacts
MODELS_DIR = Path("models_streamlit_xgb")
INDEX_CSV  = Path("index.csv")
SCALER_P   = Path("2/scaler.joblib")
SUBJECTS_P = Path("3/subjects.json")
MF_PATH    = Path("models_streamlit_mf/find-subject-score.joblib")
GGM_PATH   = Path("models_streamlit_ggm/ggm.joblib")

# Data
DATA_XLSX  = Path("Data_clean/Data_subject_complete.xlsx")
EVAL_SPLIT = "test"  # 'train' | 'val' | 'test' | ('val','test')

# Masks list (tổ hợp)
TRAIN_MASKS_CSV = Path("subject/train_masks_samples.csv")  # << dùng file đã upload
MAX_MASK_ROWS: int | None = 200    # chỉ cắt SỐ DÒNG TỔ HỢP; None = không cắt
RANDOM_MASK_SAMPLE: bool = False   # True: lấy ngẫu nhiên n mask; False: lấy n dòng đầu
RANDOM_SEED: int = 42

# Output
PREDS_CSV    = Path("Data_clean/preds_generated.csv")
METRICS_XLSX = Path("metrics.xlsx")

# Control
ONLY_SUBJECTS: list[str] | None = None   # nếu muốn giới hạn tập môn so với file subjects.json
MAX_ROWS: int | None = None              # để chạy nhanh (cắt số học sinh). KHÔNG đụng để giữ metrics chuẩn
MIN_K = 3
EPS = 1e-6
warnings.filterwarnings("ignore", category=RuntimeWarning)


# =========================
# 1) LOADERS
# =========================

def load_subjects(path: Path) -> list[str]:
    with open(path, "r", encoding="utf-8") as f:
        obj = json.load(f)
    if isinstance(obj, dict) and "subjects" in obj:
        return list(obj["subjects"])
    if isinstance(obj, list):
        return list(obj)
    raise ValueError("SUBJECTS_P must be a JSON list or a dict with key 'subjects'.")


def load_excel(path: Path) -> pd.DataFrame:
    df = pd.read_excel(path)
    need = {"EncryptedID", "split"}
    if not need.issubset(df.columns):
        raise ValueError(f"Excel must include {need}. Found: {list(df.columns)}")
    return df


def load_scaler(path: Path, train_df: pd.DataFrame, subjects: list[str]) -> tuple[dict, dict]:
    means, stds, ok = {}, {}, False
    if path.exists():
        try:
            scl = joblib.load(path)
            if isinstance(scl, dict):
                if "means" in scl and "stds" in scl:
                    means = {k: float(v) for k, v in scl["means"].items()}
                    stds  = {k: float(v) for k, v in scl["stds"].items()}
                    ok = True
                elif "mean" in scl and "scale" in scl:
                    means = {s: float(m) for s, m in zip(subjects, scl["mean"])}
                    stds  = {s: float(v if v != 0 else 1.0) for s, v in zip(subjects, scl["scale"])}
                    ok = True
            else:
                if hasattr(scl, "mean_") and hasattr(scl, "scale_"):
                    means = {s: float(m) for s, m in zip(subjects, scl.mean_)}
                    stds  = {s: float(v if v != 0 else 1.0) for s, v in zip(subjects, scl.scale_)}
                    ok = True
        except Exception as e:
            print("[Scaler] load failed:", e)
    if not ok:
        sub_train = train_df[subjects]
        means = sub_train.mean(axis=0, skipna=True).to_dict()
        stds  = sub_train.std(axis=0, ddof=0, skipna=True).replace(0, 1.0).to_dict()
        means = {k: float(v) for k, v in means.items()}
        stds  = {k: float(v) for k, v in stds.items()}
        print("⚠️ Using fallback means/stds computed from train split.")
    return means, stds


def load_xgb_index(path: Path) -> pd.DataFrame:
    idx = pd.read_csv(path)
    need = {"target", "K", "model_path"}
    if not need.issubset(idx.columns):
        raise ValueError(f"index.csv must include columns {need}. Found: {list(idx.columns)}")
    idx["model_path"] = idx["model_path"].astype(str)
    return idx


def resolve_model_path(model_path_str: str, base_dir: Path) -> Path:
    p = Path(model_path_str)
    if p.exists():
        return p
    if not p.is_absolute():
        cand = base_dir / p
        if cand.exists():
            return cand
    return cand if 'cand' in locals() else p


def load_mf(path: Path):
    return joblib.load(path) if path.exists() else None


def load_ggm(path: Path):
    return joblib.load(path) if path.exists() else None


# =========================
# 2) FEATURES
# =========================

def standardize(val: float, mu: float, sd: float) -> float:
    sd = sd if (sd and sd != 0) else 1.0
    return (val - mu) / sd


def standardize_user_numeric(user_numeric: dict, means: dict, stds: dict, subjects: list[str]) -> dict:
    out = {}
    for s in subjects:
        v = user_numeric.get(s, np.nan)
        if v is None or (isinstance(v, float) and np.isnan(v)):
            out[s] = np.nan
        else:
            out[s] = standardize(float(v), means.get(s, 0.0), stds.get(s, 1.0))
    return out


def build_xgb_features(user_numeric: dict, means: dict, stds: dict, subjects: list[str]) -> np.ndarray:
    z = standardize_user_numeric(user_numeric, means, stds, subjects)
    vals = np.array([z[s] for s in subjects], dtype=float)
    miss = (~np.isfinite(vals)).astype(float)
    vals = np.nan_to_num(vals, nan=0.0)
    return np.concatenate([vals, miss], axis=0)


# =========================
# 3) MODELS
# =========================

def select_xgb_model_path(target: str, K: int, xgb_index: pd.DataFrame) -> Path | None:
    df = xgb_index[xgb_index["target"] == target]
    if df.empty:
        return None
    if (df["K"] == K).any():
        mp = df[df["K"] == K]["model_path"].iloc[0]
        return resolve_model_path(mp, MODELS_DIR)
    df2 = df.assign(diff=(df["K"] - K).abs()).sort_values("diff")
    mp = df2.iloc[0]["model_path"]
    return resolve_model_path(mp, MODELS_DIR)


def predict_xgb_for_target(target: str, K: int, features: np.ndarray, means: dict, stds: dict, xgb_index: pd.DataFrame) -> float:
    model_path = select_xgb_model_path(target, K, xgb_index)
    if model_path is None or not model_path.exists():
        return np.nan
    try:
        model = joblib.load(model_path)
        y_std = float(model.predict(features.reshape(1, -1))[0])
        return y_std * stds[target] + means[target]
    except Exception:
        return np.nan


def predict_mf_for_target(mf_art, means, stds, user_numeric: dict, target: str, subjects: list[str]) -> float:
    if mf_art is None:
        return np.nan
    subj_mf = mf_art.get("subjects", subjects)
    # bias
    if "item_bias" in mf_art:
        b = np.array([mf_art["item_bias"].get(s, 0.0) for s in subj_mf], dtype=float)
    elif "b" in mf_art:
        b = np.array(mf_art["b"], dtype=float)
    else:
        b = np.zeros(len(subj_mf), dtype=float)
    # factors
    V = None
    for key in ("V", "item_factors", "Q"):
        if key in mf_art:
            V = np.array(mf_art[key], dtype=float)
            break
    if V is None:
        return np.nan
    lam = float(mf_art.get("lambda", mf_art.get("reg", 1.0)))
    idx_map = {s: i for i, s in enumerate(subj_mf)}
    z = standardize_user_numeric(user_numeric, means, stds, subjects)
    known_idx = [i for i, s in enumerate(subjects) if np.isfinite(z[s])]
    known_idx_mf = [idx_map[subjects[i]] for i in known_idx if subjects[i] in idx_map]
    if len(known_idx_mf) == 0:
        return np.nan
    r_known = np.array([z[subjects[i]] for i in known_idx], dtype=float)
    V_known = V[known_idx_mf, :]
    b_known = b[known_idx_mf]
    y = r_known - b_known
    A = V_known.T @ V_known + lam * np.eye(V_known.shape[1])
    u = np.linalg.pinv(A) @ (V_known.T @ y)
    if target not in idx_map:
        return np.nan
    j = idx_map[target]
    z_hat = float(b[j] + V[j, :].T @ u)
    return z_hat * stds[target] + means[target]


def predict_ggm_for_target(ggm_art, means, stds, user_numeric: dict, target: str, subjects: list[str]) -> tuple[float, float | None]:
    if ggm_art is None:
        return (np.nan, None)
    Sigma = None
    for k in ("cov", "sigma", "covariance"):
        if k in ggm_art:
            Sigma = np.array(ggm_art[k], dtype=float)
            break
    if Sigma is None:
        return (np.nan, None)
    subj = ggm_art.get("subjects", subjects)
    idx_map = {s: i for i, s in enumerate(subj)}
    if target not in idx_map:
        return (np.nan, None)
    z = standardize_user_numeric(user_numeric, means, stds, subjects)
    obs = [i for i, s in enumerate(subjects) if np.isfinite(z[s]) and s in idx_map]
    if len(obs) == 0:
        return (np.nan, None)
    t = idx_map[target]
    O = [idx_map[subjects[i]] for i in obs]
    try:
        Sigma_OO = Sigma[np.ix_(O, O)]
        Sigma_TO = Sigma[np.ix_([t], O)]
        Sigma_OT = Sigma[np.ix_(O, [t])]
        Sigma_TT = Sigma[t, t]
        z_O = np.array([z[subjects[i]] for i in obs], dtype=float).reshape(-1, 1)
        inv_Sigma_OO = np.linalg.pinv(Sigma_OO)
        mu_cond = (Sigma_TO @ inv_Sigma_OO @ z_O).ravel()[0]
        var_cond = float(Sigma_TT - (Sigma_TO @ inv_Sigma_OO @ Sigma_OT).ravel()[0])
        y_hat = mu_cond * stds[target] + means[target]
        return y_hat, max(var_cond, 0.0)
    except Exception:
        return (np.nan, None)


# =========================
# 4) BLEND
# =========================

def blend(y_xgb: float, y_mf: float, y_ggm: float, var_ggm: float | None, K: int, eps: float = 1e-6) -> tuple[float, tuple[float, float, float]]:
    if K <= 10:
        wx, wm = 0.85, 0.15
    else:
        wx, wm = 0.70, 0.30
    w_ggm = 0.0
    if var_ggm is not None and np.isfinite(var_ggm):
        conf = 1.0 / (var_ggm + eps)
        w_ggm = 0.25 * float(conf / (conf + 1.0))  # ≤ 0.25
    remain = 1.0 - w_ggm
    total = wx + wm
    wx_norm = remain * wx / total if total else 0.0
    wm_norm = remain * wm / total if total else 0.0
    parts = [("xgb", y_xgb, wx_norm), ("mf", y_mf, wm_norm), ("ggm", y_ggm, w_ggm)]
    valid = [(y, w) for _, y, w in parts if np.isfinite(y)]
    if not valid:
        return np.nan, (0.0, 0.0, 0.0)
    total_w = sum(w for _, w in valid)
    if total_w == 0:
        y = float(np.mean([y for y, _ in valid]))
        return y, (wx_norm, wm_norm, w_ggm)
    y = sum(w * y for y, w in valid) / total_w
    return y, (wx_norm, wm_norm, w_ggm)


# =========================
# 5) METRICS (FORMULA, NaN-SAFE)
# =========================

def mae(y_true, y_pred):
    y_true = np.asarray(y_true, dtype=float)
    y_pred = np.asarray(y_pred, dtype=float)
    m = np.isfinite(y_true) & np.isfinite(y_pred)
    e = y_true[m] - y_pred[m]
    return float(np.mean(np.abs(e))) if e.size else float("nan")


def rmse(y_true, y_pred):
    y_true = np.asarray(y_true, dtype=float)
    y_pred = np.asarray(y_pred, dtype=float)
    m = np.isfinite(y_true) & np.isfinite(y_pred)
    e = y_true[m] - y_pred[m]
    return float(np.sqrt(np.mean(e**2))) if e.size else float("nan")


def r2(y_true, y_pred):
    y_true = np.asarray(y_true, dtype=float)
    y_pred = np.asarray(y_pred, dtype=float)
    m = np.isfinite(y_true) & np.isfinite(y_pred)
    y = y_true[m]
    yhat = y_pred[m]
    if y.size < 2:
        return float("nan")
    ss_res = np.sum((y - yhat) ** 2)
    ss_tot = np.sum((y - np.mean(y)) ** 2)
    if ss_tot == 0:
        return 1.0 if ss_res == 0 else 0.0
    return float(1.0 - ss_res / ss_tot)


# =========================
# 6) MASK PARSER
# =========================

def parse_kept_subjects(cell: str) -> list[str]:
    s = str(cell).strip()
    # thử JSON hoặc python-literal
    for parser in (json.loads, ast.literal_eval):
        try:
            v = parser(s)
            if isinstance(v, list) and all(isinstance(x, str) for x in v):
                return [x.strip() for x in v if x and x.strip()]
        except Exception:
            pass
    # mặc định: phân tách bằng dấu phẩy
    return [x.strip() for x in s.split(",") if x.strip()]


# =========================
# 7) MAIN
# =========================
if __name__ == "__main__":
    # Load data/artifacts
    subjects_all = load_subjects(SUBJECTS_P)
    data_df = load_excel(DATA_XLSX)
    train_mask = data_df["split"].astype(str).str.lower().eq("train")
    means, stds = load_scaler(SCALER_P, data_df[train_mask], subjects_all)
    xgb_index = load_xgb_index(INDEX_CSV)
    mf_art = load_mf(MF_PATH)
    ggm_art = load_ggm(GGM_PATH)

    # Chọn split & subject list giao nhau với Excel
    spl = data_df["split"].astype(str).str.lower()
    if isinstance(EVAL_SPLIT, (list, tuple, set)):
        use_mask = spl.isin([str(s).lower() for s in EVAL_SPLIT])
    else:
        use_mask = spl.eq(str(EVAL_SPLIT).lower())
    base_df = data_df.loc[use_mask].copy()

    if MAX_ROWS is not None:
        base_df = base_df.head(int(MAX_ROWS)).copy()  # cắt HỌC SINH nếu cần (mặc định để None)

    excel_subjects = [c for c in base_df.columns if c not in {"EncryptedID", "split"}]
    subj_set = set(subjects_all) if ONLY_SUBJECTS is None else set(ONLY_SUBJECTS)
    subjects = [s for s in subjects_all if s in subj_set and s in excel_subjects]

    print(f"Loaded subjects: {len(subjects_all)} | Using: {len(subjects)} (on split={EVAL_SPLIT})")
    print(f"MF: {'OK' if mf_art is not None else 'None'} | GGM: {'OK' if ggm_art is not None else 'None'}")

    # Load masks
    masks_df = pd.read_csv(TRAIN_MASKS_CSV)
    need = {"target", "scenario", "K", "kept_subjects"}
    if not need.issubset(masks_df.columns):
        raise SystemExit(f"Mask CSV must include {need}. Found: {list(masks_df.columns)}")

    # Chỉ GIỚI HẠN SỐ DÒNG TỔ HỢP (không ảnh hưởng số lượng học sinh)
    total_masks = len(masks_df)
    if isinstance(MAX_MASK_ROWS, int) and MAX_MASK_ROWS > 0 and total_masks > MAX_MASK_ROWS:
        if RANDOM_MASK_SAMPLE:
            masks_df = masks_df.sample(n=MAX_MASK_ROWS, random_state=RANDOM_SEED).reset_index(drop=True)
            print(f"Masks limited: sampled {len(masks_df)}/{total_masks} rows (random)")
        else:
            masks_df = masks_df.head(MAX_MASK_ROWS).reset_index(drop=True)
            print(f"Masks limited: head {len(masks_df)}/{total_masks} rows")
    else:
        print(f"Masks used: {len(masks_df)} rows (no limit)")

    # Prepare predictions
    recs = []
    for _, mrow in masks_df.iterrows():
        target = str(mrow["target"])
        scenario = str(mrow["scenario"])
        K_declared = int(mrow["K"])
        keep_list = parse_kept_subjects(mrow["kept_subjects"])
        keep_set = set(keep_list)

        if target not in subjects:
            # target không nằm trong tập môn đang dùng -> bỏ qua
            continue

        # cảnh báo nếu có tên môn lạ trong kept_subjects (log nhẹ, vẫn chạy với phần giao)
        unknown = [s for s in keep_set if s not in subjects]
        if unknown:
            pass

        for _, row in base_df.iterrows():
            sid = row["EncryptedID"]

            # full user values
            user_full = {s: (None if pd.isna(row.get(s)) else float(row.get(s))) for s in subjects}
            y_true = user_full.get(target, None)
            if y_true is None or (isinstance(y_true, float) and np.isnan(y_true)):
                continue

            # mask theo kept_subjects (chỉ giữ các môn trong danh sách & khác target)
            user_numeric = dict(user_full)
            for s in subjects:
                if s == target:
                    user_numeric[s] = np.nan   # luôn mask target
                elif s not in keep_set:
                    user_numeric[s] = np.nan

            # K hiệu dụng sau khi giao với dữ liệu thật
            K_eff = int(sum(1 for s, v in user_numeric.items() if s != target and v is not None and np.isfinite(v)))
            if K_eff < MIN_K:
                continue

            feats = build_xgb_features(user_numeric, means, stds, subjects)
            yx = predict_xgb_for_target(target, K_eff, feats, means, stds, xgb_index)
            ym = predict_mf_for_target(mf_art, means, stds, user_numeric, target, subjects)
            yg, varg = predict_ggm_for_target(ggm_art, means, stds, user_numeric, target, subjects)
            yh, (wx_eff, wm_eff, wg_eff) = blend(yx, ym, yg, varg, K_eff, eps=EPS)

            recs.append({
                "scenario": scenario,
                "target": target,
                "K_declared": K_declared,
                "K_eff": K_eff,
                "EncryptedID": sid,
                "y_true": y_true,
                "y_pred_xgb": yx,
                "y_pred_mf": ym,
                "y_pred_ggm": yg,
                "y_pred_hyb": yh,
                "w_xgb_eff": wx_eff,
                "w_mf_eff": wm_eff,
                "w_ggm_eff": wg_eff,
                "var_ggm": varg,
            })

    preds = pd.DataFrame.from_records(recs)
    preds.to_csv(PREDS_CSV, index=False)
    print(f"✅ Saved predictions -> {PREDS_CSV} (rows={len(preds)})")
    if preds.empty:
        raise SystemExit("No predictions generated from masks. Check mask CSV and subjects mapping.")

    # =========================
    # METRICS
    # =========================
    pred_cols = [c for c in preds.columns if c.startswith("y_pred_")]

    # Per-mask metrics (mỗi tổ hợp 1 hàng)
    permask_rows = []
    for (sc, tgt, Kd), g in preds.groupby(["scenario", "target", "K_declared"], dropna=False):
        y_true = g["y_true"].to_numpy(dtype=float)
        for col in sorted(pred_cols):
            y_pred = g[col].to_numpy(dtype=float)
            permask_rows.append({
                "scenario": sc,
                "target": tgt,
                "K_declared": Kd,
                "Model": col.replace("y_pred_", ""),
                "MAE": mae(y_true, y_pred),
                "RMSE": rmse(y_true, y_pred),
                "R2": r2(y_true, y_pred),
                "N": int(len(g)),
            })
    per_mask_df = pd.DataFrame(permask_rows).sort_values(["scenario", "target", "K_declared", "Model"]).reset_index(drop=True)

    # Macro-avg by scenario
    macro_rows = []
    for sc, g in per_mask_df.groupby("scenario", dropna=False):
        for model, gm in g.groupby("Model"):
            macro_rows.append({
                "Scope": f"macro_by_scenario",
                "scenario": sc,
                "Model": model,
                "MAE_mean": gm["MAE"].mean(),
                "MAE_std": gm["MAE"].std(ddof=0),
                "RMSE_mean": gm["RMSE"].mean(),
                "RMSE_std": gm["RMSE"].std(ddof=0),
                "R2_mean": gm["R2"].mean(),
                "R2_std": gm["R2"].std(ddof=0),
                "Runs": int(gm.shape[0]),
            })
    macro_df = pd.DataFrame(macro_rows).sort_values(["scenario", "Model"]).reset_index(drop=True)

    # Macro-avg by K<16 vs K≥16 (dựa K_eff)
    preds["K_group"] = np.where(preds["K_eff"] < 16, "K<16", "K≥16")
    permask_k_rows = []
    for (kg, sc, tgt), g in preds.groupby(["K_group", "scenario", "target"], dropna=False):
        y_true = g["y_true"].to_numpy(dtype=float)
        for col in sorted(pred_cols):
            y_pred = g[col].to_numpy(dtype=float)
            permask_k_rows.append({
                "K_group": kg,
                "scenario": sc,
                "target": tgt,
                "Model": col.replace("y_pred_", ""),
                "MAE": mae(y_true, y_pred),
                "RMSE": rmse(y_true, y_pred),
                "R2": r2(y_true, y_pred),
                "N": int(len(g)),
            })
    per_mask_k_df = pd.DataFrame(permask_k_rows)

    macro_k_rows = []
    for (kg, model), g in per_mask_k_df.groupby(["K_group", "Model"], dropna=False):
        macro_k_rows.append({
            "Scope": f"macro_{kg}",
            "Model": model,
            "MAE_mean": g["MAE"].mean(),
            "MAE_std": g["MAE"].std(ddof=0),
            "RMSE_mean": g["RMSE"].mean(),
            "RMSE_std": g["RMSE"].std(ddof=0),
            "R2_mean": g["R2"].mean(),
            "R2_std": g["R2"].std(ddof=0),
            "Runs": int(g.shape[0]),
        })
    macro_k_df = pd.DataFrame(macro_k_rows).sort_values(["Scope", "Model"]).reset_index(drop=True)

    # Micro-avg (concatenate) cho toàn bộ, và cho từng K_group
    def micro_metrics(df: pd.DataFrame, label: str) -> pd.DataFrame:
        rows = []
        y_true = df["y_true"].to_numpy(dtype=float)
        for col in sorted(pred_cols):
            y_pred = df[col].to_numpy(dtype=float)
            rows.append({
                "Scope": label,
                "Model": col.replace("y_pred_", ""),
                "MAE": mae(y_true, y_pred),
                "RMSE": rmse(y_true, y_pred),
                "R2": r2(y_true, y_pred),
                "N": int(len(df)),
            })
        return pd.DataFrame(rows).sort_values(["Scope", "Model"]).reset_index(drop=True)

    micro_all_df  = micro_metrics(preds, "micro_overall")
    micro_lt16_df = micro_metrics(preds[preds["K_eff"] < 16], "micro_K<16") if (preds["K_eff"] < 16).any() else pd.DataFrame()
    micro_ge16_df = micro_metrics(preds[preds["K_eff"] >= 16], "micro_K≥16") if (preds["K_eff"] >= 16).any() else pd.DataFrame()

    # Print tóm tắt
    print("\n=== MICRO (overall) ===")
    print(micro_all_df.to_string(index=False))
    if not micro_lt16_df.empty:
        print("\n=== MICRO (K < 16) ===")
        print(micro_lt16_df.to_string(index=False))
    if not micro_ge16_df.empty:
        print("\n=== MICRO (K ≥ 16) ===")
        print(micro_ge16_df.to_string(index=False))

    # Export Excel
    with pd.ExcelWriter(METRICS_XLSX, engine="xlsxwriter") as writer:
        preds.to_excel(writer, sheet_name="preds", index=False)
        per_mask_df.to_excel(writer, sheet_name="per_mask", index=False)
        macro_df.to_excel(writer, sheet_name="macro_by_scenario", index=False)
        macro_k_df.to_excel(writer, sheet_name="macro_by_Kgroup", index=False)
        micro_all_df.to_excel(writer, sheet_name="micro_overall", index=False)
        if not micro_lt16_df.empty:
            micro_lt16_df.to_excel(writer, sheet_name="micro_K_lt16", index=False)
        if not micro_ge16_df.empty:
            micro_ge16_df.to_excel(writer, sheet_name="micro_K_ge16", index=False)

    print(f"\n✅ Saved metrics -> {METRICS_XLSX}")


Loaded subjects: 31 | Using: 31 (on split=test)
MF: OK | GGM: OK
Masks limited: head 200/124000 rows
✅ Saved predictions -> Data_clean\preds_generated.csv (rows=27200)

=== MICRO (overall) ===
        Scope Model      MAE     RMSE        R2     N
micro_overall   ggm 0.684488 0.822441  0.057633 27200
micro_overall   hyb 0.687239 0.825998  0.049463 27200
micro_overall    mf 0.864180 1.063842 -0.576756 27200
micro_overall   xgb 0.692089 0.834827  0.029036 27200

=== MICRO (K < 16) ===
     Scope Model      MAE     RMSE        R2     N
micro_K<16   ggm 0.684488 0.822441  0.057633 27200
micro_K<16   hyb 0.687239 0.825998  0.049463 27200
micro_K<16    mf 0.864180 1.063842 -0.576756 27200
micro_K<16   xgb 0.692089 0.834827  0.029036 27200

✅ Saved metrics -> metrics.xlsx
